<p align="center">
  <img src="https://i.ytimg.com/vi/Wm8ftqDZUVk/maxresdefault.jpg" alt="FIUBA" width="25%"/>
  </p>
  
# **Trabajo Práctico 2: Críticas Cinematográficas**
### **Grupo**: 11 - Los Pandas 🐼
### **Cuatrimestre**: 2ºC 2023
### **Corrector**: Mateo Suster
### **Integrantes**:
- ### 106861 - Labollita, Francisco
- ### 102312 - Mundani Vegega, Ezequiel
- ###  97263 - Otegui, Matías Iñaki

# Modelo Random Forest

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
reviews = pd.read_csv('train_clean.csv')

## Implementación del bag of words

In [ ]:
vectorizerTotal = CountVectorizer(strip_accents='unicode', dtype='uint16')
vectorizerTotal.fit_transform(reviews['review_es'])

# Primeros 20 elementos
print(vectorizerTotal.get_feature_names_out()[:20])
# Elementos del medio
print(vectorizerTotal.get_feature_names_out()[10000:10020])
# Últimos 20 elementos
print(vectorizerTotal.get_feature_names_out()[-20:])

Se ve que varias "palabras" serán números, algunas tendrán símbolos no pertenecientes al alfabeto español y también se comprueba que están palabras españolas.

## Feature engineering del bag of words

En primer lugar, siendo que todas las palabras que inician una oración empiezan en mayúscula, se hará que todas las letras de palabras con una sola mayúscula sean transformadas a minúsculas. De tal manera que en el siguiente ejemplo, las dos variaciones de hermosa sean una misma palabra: "Hermosa película" y "Esta película es hermosa".

In [ ]:
matrizApariciones = vectorizerTotal.fit_transform(reviews['review_es'])

In [ ]:
matrizSiAparece = matrizApariciones.toarray()
matrizApariciones = matrizApariciones.toarray()

In [ ]:
matrizSiAparece[matrizSiAparece > 0] = 1

In [ ]:
words_df = pd.DataFrame()
words_df['Palabra'] = vectorizerTotal.get_feature_names_out()
words_df['Apariciones Totales'] = matrizApariciones.sum(axis=0).tolist() #Cuántas veces aparece la palabra
words_df['Apariciones'] = matrizSiAparece.sum(axis=0).tolist()           #En cuántas reviews aparece la palabra

In [ ]:
#Se cuentan en cuántas reviews positivas aparece cada palabra
listaAparicionesPositivas = np.zeros(shape=len(matrizSiAparece[0])).astype('int32')
for i in range(reviews.shape[0]):
    if (reviews.iloc[i]['sentimiento'] == 'positivo'):
        listaAparicionesPositivas += matrizSiAparece[i]

#Se cuentan en cuántas reviews negativas aparece cada palabra
listaAparicionesNegativas = np.zeros(shape=len(matrizSiAparece[0])).astype('int32')
for i in range(reviews.shape[0]):
    if (reviews.iloc[i]['sentimiento'] == 'negativo'):
        listaAparicionesNegativas += matrizSiAparece[i]

In [ ]:
words_df['Apariciones positivas'] = listaAparicionesPositivas
words_df['Apariciones negativas'] = listaAparicionesNegativas
words_df['Fracción apariciones positivas'] = words_df['Apariciones positivas'] / words_df['Apariciones']
words_df['Fracción apariciones negativas'] = words_df['Apariciones negativas'] / words_df['Apariciones']
words_df['Tasa de positividad'] = (words_df['Apariciones positivas'] - words_df['Apariciones negativas']) / words_df['Apariciones']
words_df.sort_values(by='Apariciones', inplace=True, ascending=False)
words_df.head(10)

### Se entrena un módelo posible de Random Forest

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(
    reviews['review_es'], reviews['sentimiento'], test_size=0.2, random_state=42)

# Crear una matriz de términos de documento utilizando CountVectorizer
vectorizer = CountVectorizer()
x_train_counts = vectorizer.fit_transform(x_train)

# Entrenar el modelo de Random Forest
model = RandomForestClassifier(max_features='sqrt',
                             random_state=2,
                             n_jobs=-1,
                             criterion="entropy",
                             n_estimators=50)

model.fit(x_train_counts, y_train)

# Transformar los datos de prueba y hacer predicciones
x_test_counts = vectorizer.transform(x_test)
y_test_pred = model.predict(x_test_counts)

In [ ]:
tabla = confusion_matrix(y_test, y_test_pred)
print("Matriz de confusión de los datos de prueba")
sns.heatmap(tabla, cmap='GnBu', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

In [ ]:
accuracy = accuracy_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print("Cálculo de las métricas en el conjunto de pruebas")
print("Accuracy: "+str(accuracy))
print("Recall: "+str(recall))
print("Precision: "+str(precision))
print("F1 Score: "+str(f1))

## Se buscan los hiperparámetros con GridSearch CV

Se realizó una busqueda de los mejores parámetros con el método GridSearchCV proporcionando varios parámetros posibles basados en lo estudiado en las clases prácticas.

Se obtuvo que los mejores hiperparámetros (utilizando el F1 score como métrica) para este caso son:

- *min_samples_leaf* = $X$
- Criterio = XXXX
- *min_samples_split* = X
- *n_estimators* = XX

Logrando un F1 score de 0.XXX, mayor en comparación al primer árbol obtenido que fue de 0.XXX

In [ ]:
rf_cv = RandomForestClassifier(
    max_features='sqrt', oob_score=True, random_state=1, n_jobs=-1)

param_grid = {"criterion": ["gini", "entropy"],
              "min_samples_leaf": [1, 5, 10],
              "min_samples_split": [2, 4, 10, 12, 16],
              "n_estimators": [10, 20, 50]}


metricas = ['accuracy', 'f1', 'roc_auc']

gs_multimetrica = GridSearchCV(estimator=rf_cv,
                               param_grid=param_grid,
                               scoring=metricas,
                               refit=False,
                               cv=5,
                               n_jobs=-1)

gs_multimetrica_fit = gs_multimetrica.fit(X=x_train, y=y_train)

params_elegidos = gs_multimetrica_fit.cv_results_[
    'params'][np.argmax(gs_multimetrica_fit.cv_results_['mean_test_accuracy'])]
params_elegidos

In [ ]:
labels = [key for key in gs_multimetrica_fit.cv_results_.keys()
          if ("mean_test" in key)]

for k in labels:
    plt.plot(gs_multimetrica_fit.cv_results_[
             k], linestyle='--', linewidth=0.8, marker='o', markersize=2)
    x_linea = np.argmax(gs_multimetrica_fit.cv_results_[k])
    plt.axvline(x_linea, linestyle='--', linewidth=0.8, color='grey')

plt.xlabel("modelo", fontsize=10)
plt.ylabel("métrica", fontsize=10)
plt.legend(labels)
plt.show()

In [ ]:
rfc_multimetrica = RandomForestClassifier(criterion='gini',
                                          min_samples_leaf=1,
                                          min_samples_split=4,
                                          n_estimators=50,
                                          oob_score=True,
                                          random_state=2,
                                          n_jobs=-1)

model = rfc_multimetrica.fit(X=x_train, y=y_train)

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

In [ ]:
print("Matriz de confusión de los datos de prueba")
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

In [ ]:
accuracy = accuracy_score(y_train, y_train_pred)
recall = recall_score(y_train, y_train_pred)
f1 = f1_score(y_train, y_train_pred)
precision = precision_score(y_train, y_train_pred)

print("Cálculo de las métricas en el conjunto de entrenamiento")
print("Accuracy: ", round(accuracy, 3))
print("Recall: ", round(recall, 3))
print("Precision: ", round(precision, 3))
print("F1 score: ", round(f1, 3))

print(classification_report(y_train, y_train_pred))

## Predicción del conjunto test

In [ ]:
test_df = pd.read_csv('test_clean.csv')

x_test_counts = vectorizer.transform(test_df['review_es'])
y_pred_test = model.predict(x_test_counts)

test_df['sentimiento'] = y_pred_test

test_df.drop("review_es", axis=1, inplace=True)
test_df.to_csv('sample_solution.csv', index=False)